`Module 2`
# Extracting Information from the Content

## Parse columns from raw text

In [ ]:
-- Extract PRODUCT name, DATE, and CUSTOMER_REVIEW from the LAYOUT column
SELECT
    filename,
    REGEXP_SUBSTR(extracted_content:content, 'Product: (.*?)\\n', 1, 1, 'e') as product,
    TO_DATE(REGEXP_SUBSTR(extracted_content:content, 'Date: (202[0-9]-[0-9]{2}-[0-9]{2})', 1, 1, 'e')) as review_date,
    CASE 
        WHEN POSITION('Customer Review' IN extracted_content:content) > 0 THEN
            SUBSTRING(extracted_content:content, POSITION('Customer Review' IN extracted_content:content) + LENGTH('Customer Review'))
        ELSE NULL
    END as review_text,
    CAST(CONCAT('2', REGEXP_SUBSTR(filename, '\\\d+')) AS INTEGER) AS order_id,
FROM raw_customer_reviews;

## Write to Snowflake table

In [ ]:
-- Create a table to store the parsed and structured document content with order_id
CREATE OR REPLACE TABLE avalanche_db.avalanche_schema.customer_reviews (
    filename VARCHAR,
    product VARCHAR,
    review_date DATE,
    review_text TEXT,
    order_id INTEGER
);

-- Insert the parsed content, including order_id derived from filename
INSERT INTO avalanche_db.avalanche_schema.customer_reviews (filename, product, review_date, review_text, order_id)
SELECT
    *
FROM {{cell3}};